In [29]:
# Function to generate Zadoff-Chu sequence
function zadoff_chu_sequence(u, N)
    return exp.(-1.0im * π * u * (0:N-1) .* (1:N) / N)
end

zadoff_chu_sequence (generic function with 1 method)

In [30]:
# Function to perform frequency-domain correlation
function freq_domain_correlation(rx_signal, u, Ncs, nuind)
    xuv = zadoff_chu_sequence(u, L)
    xuv_shifted = [xuv[1 + rem(n + (Ncs * nuind), L)] for n in 0:L-1]
    yuv = fft(xuv_shifted)
    
    rxsig_freq = fft(rx_signal)
    correlation = ifft(rxsig_freq .* conj.(yuv))
    
    return correlation
end

freq_domain_correlation (generic function with 1 method)

In [31]:
# Function to estimate time-delay
function estimate_time_delay(correlation)
    max_value, max_index = findmax(abs.(correlation))
    return max_index - 1
end

estimate_time_delay (generic function with 1 method)

In [32]:
using FFTW
using DSP

In [33]:
## Load the useful math operations
include("../modules/operations.jl");

In [34]:
# this is a 5G PRACH format 0 transmission
L=839

# this is number of samples per bin, floor(L/Ncs) gives the number of cyclic shifts, see below
Ncs=26
# this is the FFT size for the generation/reception of PRACH
N=49152
# this is the length of the cyclic prefix for PRACH
Ncp=6336

6336

In [35]:
# 6-bit data messages for 3 transmitters / UEs
preamble_index1=63;
preamble_index2=31;
preamble_index3=11;

# up to 6 Zadoff-Chu root sequences for this format
utab=[129 710 140 699 120 719]

1×6 Matrix{Int64}:
 129  710  140  699  120  719

In [36]:
# number of cyclic shifts
nshifts = floor(Int, L / Ncs)
# number of Zadoff-Chu sequences required
nseq = ceil(Int, 64 / nshifts)


2

In [37]:
# index of the preamble sequence to use
uind1 = div(preamble_index1, nshifts)
uind2 = div(preamble_index2, nshifts)
uind3 = div(preamble_index3, nshifts)

# index of cyclic shift to use
nuind1 = rem(preamble_index1, nshifts)
nuind2 = rem(preamble_index2, nshifts)
nuind3 = rem(preamble_index3, nshifts)

# Check if indices are within the range of utab
if (uind1 >= length(utab) || uind2 >= length(utab) || uind3 >= length(utab))
    println("ERROR tab length ", length(utab), " : ", uind1, " ", uind2, " ", uind3)
end

In [38]:
# These are the Zadoff-Chu Sequence generators (time-domain) 
# for the 3 transmitters
xu1 = exp.(-im * π * utab[1 + uind1] * collect(0:838) .* collect(1:839) / 839)
xu2 = exp.(-im * π * utab[1 + uind2] * collect(0:838) .* collect(1:839) / 839)
xu3 = exp.(-im * π * utab[1 + uind3] * collect(0:838) .* collect(1:839) / 839)
@show size(xu1), size(xu2), size(xu3)

(size(xu1), size(xu2), size(xu3)) = ((839,), (839,), (839,))


((839,), (839,), (839,))

In [39]:
# implement cyclic-shifts
# Note: In practice, it's not common to do cyclic shifts in the time-domain and then perform FFT.
# There is a way to compute the Fourier transform directly and then perform the cyclic shift by a multiplication of a phasor in the frequency-domain.

xuv1 = copy(xu1)
xuv2 = copy(xu2)
xuv3 = copy(xu3)

for n in 0:838
    xuv1[n + 1] = xu1[1 + rem(n + (Ncs * nuind1), 839)]
    yuv1 = fft(xuv1)
    xuv2[n + 1] = xu2[1 + rem(n + (Ncs * nuind2), 839)]
    yuv2 = fft(xuv2)
    xuv3[n + 1] = xu3[1 + rem(n + (Ncs * nuind3), 839)]
    yuv3 = fft(xuv3)
end

In [40]:
# put the PRACH in the lowest frequency (positive) subcarriers starting at carrier 7
Xuv1 = zeros(Complex{Float64}, 1, 49152)
Xuv1[8:846] .= yuv1
Xuv2 = zeros(Complex{Float64}, 1, 49152)
Xuv2[8:846] .= yuv2
Xuv3 = zeros(Complex{Float64}, 1, 49152)
Xuv3[8:846] .= yuv3;

In [41]:
# bring to time-domain
xuv1_49152 = ifft(Xuv1)
xuv2_49152 = ifft(Xuv2)
xuv3_49152 = ifft(Xuv3)
@show size(xuv1_49152), size(xuv2_49152), size(xuv3_49152)

(size(xuv1_49152), size(xuv2_49152), size(xuv3_49152)) = ((1, 49152), (1, 49152), (1, 49152))


((1, 49152), (1, 49152), (1, 49152))

In [42]:
using Plots
using LinearAlgebra

In [43]:
# add cyclic prefix
xuv1_49152 = xuv1_49152[(49152-6335):end] ⧺ xuv1_49152
# xuv2_49152 = xuv2_49152[(end-6335):end] ⧺ xuv2_49152
# xuv3_49152 = xuv3_49152[(end-6335):end] ⧺ xuv3_49152

LoadError: DimensionMismatch: number of columns of each array must match (got (1, 49152))

In [57]:
# Receiver for the first transmitter (UE)
correlation1 = freq_domain_correlation(rxsig3_noiseandchannel, utab[1 + uind1], Ncs, nuind1)
delay_estimate1 = estimate_time_delay(correlation1)

println("Time Delay Estimate for UE1: ", delay_estimate1)



LoadError: UndefVarError: `rxsig3_noiseandchannel` not defined

In [ ]:
# Receiver for the second transmitter (UE)
correlation2 = freq_domain_correlation(rxsig4_noiseandchannel, utab[1 + uind2], Ncs, nuind2)
delay_estimate2 = estimate_time_delay(correlation2)

println("Time Delay Estimate for UE2: ", delay_estimate2)
